<h1><span style="color:red">Re-coding ordinal variables</span></h1>

Using this notebook, you can select ordinal variables and prepend numbers to values. You will have an option to process a survey file received from the current SuAVE application, or import a local CSV file. Then the notebook will let you create a new SuAVE survey with the updated survey file. 

Author: Iakov Vasilyev


## 1. Retrieve survey parameters from the URL

In [ ]:
%%javascript
function getQueryStringValue (key)
{  
    return unescape(window.location.search.replace(new RegExp("^(?:.*[&\\?]" + escape(key).replace(/[\.\+\*]/g, "\\$&") + "(?:\\=([^&]*))?)?.*$", "i"), "$1"));
}
IPython.notebook.kernel.execute("survey_url='".concat(getQueryStringValue("surveyurl")).concat("'"));
IPython.notebook.kernel.execute("views='".concat(getQueryStringValue("views")).concat("'"));
IPython.notebook.kernel.execute("view='".concat(getQueryStringValue("view")).concat("'"));
IPython.notebook.kernel.execute("user='".concat(getQueryStringValue("user")).concat("'"));
IPython.notebook.kernel.execute("csv_file='".concat(getQueryStringValue("csv")).concat("'")); 
IPython.notebook.kernel.execute("dzc_file='".concat(getQueryStringValue("dzc")).concat("'")); 
IPython.notebook.kernel.execute("params='".concat(getQueryStringValue("params")).concat("'")); 
IPython.notebook.kernel.execute("active_object='".concat(getQueryStringValue("activeobject")).concat("'")); 
IPython.notebook.kernel.execute("full_notebook_url='" + window.location + "'"); 

## 2. Import libraries

In [ ]:
# common imports
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import Markdown, display

import pandas as pd
pd.set_option('display.max_colwidth', 0)
    
import numpy as np
import panel as pn

pn.extension('tabulator')
def printmd(string):
    display(Markdown(string))

absolutePath = "../../temp_csvs/"

# local imports
import sys
sys.path.insert(1, '../../helpers')
import panel_libs as panellibs
import suave_integration as suaveint

url_partitioned = full_notebook_url.partition('/operations')
base_url = url_partitioned[0];

import re

## 3. Select a survey file from SuAVE or import a local CSV file

In [ ]:
data_select = pn.widgets.RadioBoxGroup(name='Select notebook', options=['Load survey file from SuAVE', 
                                                                        'Import a local CSV file'], 
                                       inline=False)
data_select

In [ ]:
data_input = pn.widgets.FileInput()
    
def check_selection():
    if data_select.value == 'Load survey file from SuAVE':
        global fname
        fname = absolutePath + csv_file
        printmd("<b><span style='color:red; font-size: 200%;'>Current SuAVE survey will be loaded. Continue to step 4.</span></b>")

    else:
        message = pn.pane.HTML("<b><span style='color:red; font-size: 200%;'>Upload data and continue to step 4.</span><br><span style='font-size: 150%;'>IMPORTANT: The local CSV file should not have SuAVE-specific variable names!</span></b>", width=700)
        return pn.Column(message, data_input)
    
check_selection()

## 4. Visualize the data and assign codes for values of ordinal variables

In [ ]:
if not pd.isnull(data_input.filename):
    fname = absolutePath + data_input.filename
    data_input.save(fname)

# df = extract_data(fname).fillna('')
df = panellibs.extract_data(fname)

panellibs.slider(df)

In [ ]:
# defining the panel
            
likcols = df.columns.tolist()
# remove any variable names with qualifiers
likcols = [x for x in likcols if '#' not in x]
# remove variables that have less than 3 or more than 10 unique values
likcols = [x for x in likcols if df[x].nunique() > 2 and df[x].nunique() < 11]
    
if 'updated_df' in globals():
    global updated_df
    updated_df = df.copy()
        
left_text = pn.Row("####Select Ordinal Variables", margin=(0,0,-15,270))
binary_selector = pn.widgets.CrossSelector(options=list(df.columns), width=630)
left_panel = pn.Column(left_text, binary_selector, css_classes=['widget-box'], margin=(0,30,0,0))
full_display = pn.Column(left_panel)
full_display

In [ ]:
#list of columns of widgets to disply
list_displays = []
# list of widgets with the selected values
list_widgets = []
ordinal_columns = binary_selector.value
#loop through user selected columns fromt the previous cell
for i in ordinal_columns:
    if i == ordinal_columns[0]:
        left_text = pn.Row("####Select the values in each column in ascending order.", margin=(0,0,0,170))
        left_text2 = pn.Row("####For example, 1 should be at the top of the list and 10 should be at the bottom.", margin=(0,0,0,170))
    binary_selector = pn.widgets.CrossSelector(options=list(df[i].unique()), width=700, definition_order = False)
    list_widgets.append(binary_selector)
    if i == ordinal_columns[0]:
        list_displays.append(pn.Column(left_text, left_text2, binary_selector, css_classes=['widget-box'], margin=(0,50,0,0)))
    else: 
        list_displays.append(pn.Column(binary_selector, css_classes=['widget-box'], margin=(0,50,0,0)))
display(*list_displays)

In [ ]:
try:
    for i in ordinal_columns:
        temp = {}
        for j in list_widgets:
            count = 0
            for k in j.value:
                temp[k] = count
                count = count + 1
        updated_df[i+" ordinal#hiddenmore"] =  updated_df[i].map(temp).fillna(updated_df[i])
except:
    updated_df[i+" ordinal"] =  updated_df[i].map(temp).fillna(updated_df[i])[i]
    printmd("<b><span style='color:red; font-size: 150%;'>Please make sure all the values in the columns are selected in the correct order in the cell above </span></b>")
    

## 5. Generate a new survey and open it in SuAVE

In [ ]:
if data_select.value == 'Import a local CSV file':
    csv_file = data_input.filename
    dzc_file = ''
    
new_file = suaveint.save_csv_file(updated_df, absolutePath, csv_file)

In [ ]:
#Input survey name

import ipywidgets as widgets
from IPython.display import display

input_text = widgets.Text(placeholder='Enter Survey Name...')
output_text = widgets.Text()

def bind_input_to_output(sender):
    output_text.value = input_text.value

# Tell the text input widget to call bind_input_to_output() on submit
input_text.on_submit(bind_input_to_output)

printmd("<b><span style='color:red'>Input survey name here, press Enter, and then run the next cell:</span></b>")
# Display input text box widget for input
display(input_text)

display(output_text)

In [ ]:
#Print survey name
survey_name = output_text.value
printmd("<b><span style='color:red'>Survey Name is: </span></b>" + survey_name)

In [ ]:
suaveint.create_survey(survey_url,new_file, survey_name, dzc_file, user, csv_file, view, views, data_select.value)